Ling 406 - Term Project - Marco Fonseca (movie reviews)

The code below impots the necessary libraries and then saves the positive and negative movie reviews in different variables. This code assumes that the folder "tex_sentoken" is in the directory. 

In [299]:
import nltk
import random
import glob
import os
import re
import itertools


neg_list = glob.glob(os.path.join(os.getcwd(), "txt_sentoken/neg", "*.txt"))
pos_list = glob.glob(os.path.join(os.getcwd(), "txt_sentoken/pos", "*.txt"))

neg = []
pos = []

for file_path in neg_list:
    with open(file_path) as f_input:
        neg.append(f_input.read())


for file_path in pos_list:
    with open(file_path) as f_input:
        pos.append(f_input.read())

The code below tokenizes the positive reviews and the negative reviews. 

In [300]:
tokens_neg = [nltk.word_tokenize(i) for i in neg]
tokens_pos = [nltk.word_tokenize(i) for i in pos]

The code belows adds the word "neg" after all of the negative reviews and the word "pos" after all of the positive reviews. Then, it creates two variables to be used later. One variable contains all negative and positive reviews, plus its tag as "pos" or "neg". This variable is then randomized. The determined features will be searched in this tagged variabl. After that, another variable created "all words" is created. This variable does not contain the tags and will be used in order to create the features for the trainig data. 

In [301]:
out_neg = []
for l in tokens_neg: 
        tuple = ( l , "neg" ) 
        out_neg.append(tuple)


out_pos = []
for l in tokens_pos: 
        tuple = ( l , "pos" ) 
        out_pos.append(tuple)
        
pos_neg = out_pos + out_neg
random.shuffle(pos_neg)

all_words = tokens_pos + tokens_neg


The code below creates a frequency distribution of all words that will be used in order to create the features for the training data. 

In [302]:
flat_list = [item for sublist in all_words for item in sublist]
freq_dist = nltk.FreqDist(flat_list)

The code belows determines the baseline and the features to be used in this model. The baseline is comprised of unigrams. The second feature is comprised of bigrams. The third feature removes punctiation and preposition, leaving only content words. The last feature contains bigrams and only content words. 

In [303]:
baseline = list(freq_dist.keys())
bigram_features = list(nltk.bigrams(baseline)) #2nd set of features
remove = {":", ",", ".", ";", "?", "!", "(", ")", "'",  "to", "a", "the", "an", "on", "at", "in"}
content_features = [e for e in baseline if e not in remove] #3rd set of features
bigrams_content_features =  list(nltk.bigrams(content_features))#4th set of features. 


The code below creates one function to find each one of the features in the document. Since there are three features plus the baseline, 4 functions are created. Varibles containing the output of each of of the features are then created. 

In [304]:
def find_features_baseline(document):
    words = set(document)
    features = {}
    for w in baseline:
        features[w] = (w in words)
    return features 

def find_features_bigrams(document):
    words = set(document)
    features = {}
    for w in bigram_features:
        features[w] = (w in words)    
    return features

def find_features_content(document):
    words = set(document)
    features = {}
    for w in content_features:
        features[w] = (w in words)    
    return features

def find_features_bigrams_content(document):
    words = set(document)
    features = {}
    for w in bigrams_content_features:
        features[w] = (w in words)    
    return features

featuresets_baseline = [(find_features_baseline(rev), category) for (rev, category) in pos_neg]
featuresets_bigrams = [(find_features_bigrams(rev), category) for (rev, category) in pos_neg]
featuresets_content = [(find_features_content(rev), category) for (rev, category) in pos_neg]
featuresets_bigrams_content = [(find_features_bigrams_content(rev), category) for (rev, category) in pos_neg]

The code below creates the training and test data for the baseline. It was chosen to use 20% of the data for the training data and 80% of it for the test data. It also prints the accuracy for the baseline. 

In [305]:
training_set_baseline = featuresets_baseline[:1600]
testing_set_baseline = featuresets_baseline[1600:]
classifier_baseline = nltk.NaiveBayesClassifier.train(training_set_baseline)
print("Naive Bayes Algorithm accuracy of baseline:", (nltk.classify.accuracy(classifier_baseline, testing_set_baseline)))


Naive Bayes Algorithm accuracy of baseline: 0.8054862842892768


The code below creates the training and test data for the bigram feature set. It was chosen to use 20% of the data for the training data and 80% of it for the test data. It also prints the accuracy for this feature. 

In [306]:
training_set_bigrams = featuresets_bigrams[:1600]
testing_set_bigrams = featuresets_bigrams[1600:]
classifier_bigrams = nltk.NaiveBayesClassifier.train(training_set_bigrams)
print("Naive Bayes Algorithm accuracy of bigrams:", (nltk.classify.accuracy(classifier_bigrams, testing_set_bigrams)))

Naive Bayes Algorithm accuracy of bigrams: 0.4688279301745636


The code below creates the training and test data for the content words feature set. It was chosen to use 20% of the data for the training data and 80% of it for the test data. It also prints the accuracy for this feature. 

In [307]:
training_set_content = featuresets_content[:1600]
testing_set_content = featuresets_content[1600:]
classifier_content = nltk.NaiveBayesClassifier.train(training_set_content)
print("Naive Bayes Algorithm accuracy of content words:", (nltk.classify.accuracy(classifier_content, testing_set_content)))


Naive Bayes Algorithm accuracy of content words: 0.8029925187032418


The code below creates the training and test data for the bigram feature set only with content words. It was chosen to use 20% of the data for the training data and 80% of it for the test data. It also prints the accuracy for this feature. 

In [308]:
training_set_bigrams_content = featuresets_bigrams_content[:1600]
testing_set_bigrams_content = featuresets_bigrams_content[1600:]
classifier_bigrams_content = nltk.NaiveBayesClassifier.train(training_set_bigrams_content)
print("Naive Bayes Algorithm accuracy of bigrams (only content words)", (nltk.classify.accuracy(classifier_bigrams_content, testing_set_bigrams_content)))


Naive Bayes Algorithm accuracy of bigrams (only content words) 0.4688279301745636
